In [1]:
# Import the strategies for comparing

from src.strategies import preferFlush, preferFlushGroups, preferGroupFlushStraight, preferGroups, preferGroupsFlush, preferGroupStraightFlush, preferStraightFlushGroup, preferFlushStraightGroup, chooseLeft
allStrategies = [preferFlush, preferFlushGroups, preferGroupFlushStraight, preferGroups, preferGroupsFlush, preferGroupStraightFlush, preferFlushStraightGroup, preferStraightFlushGroup, chooseLeft]

ITERATIONS = 10000

In [2]:
import itertools
pairings = itertools.combinations(allStrategies, 2)

resultsTable = []

winners = []

def getPrettyName(nm):
    return nm.split(".")[-1]

for pairing in pairings:
    s1 = pairing[0]
    s2 = pairing[1]
    score = headToHead.head2Head(s1, s2, ITERATIONS)
    winner = s2
    loser = s1
    if score > 0:
        winner = s1
        loser = s2
        
    #print("%s beat %s" % (getPrettyName(winner.__name__), getPrettyName(loser.__name__)))
    winners.append(winner)

    winPercentage = int((((ITERATIONS - abs(score)) / 2) + abs(score) ) / ITERATIONS * 100)
    resultsTable.append([getPrettyName(winner.__name__), getPrettyName(loser.__name__), abs(winPercentage)])

results = {}
for strat in set(allStrategies):
    results[getPrettyName(strat.__name__)] = winners.count(strat)

In [ ]:
import pandas as pd
pd.DataFrame(resultsTable, columns=["Winner", "Loser", "%"]).sort_values('%')


In [ ]:
def most_common(lst):
    return max(set(lst), key=lst.count)

print("The winner is %s" % (getPrettyName(most_common(winners).__name__)))